In [2]:
#Install required packages 
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz #small model
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_lg-0.5.4.tar.gz #large model
!pip install spacy
!pip install nltk
!pip install gensim
!pip install numpy
!pip install pyLDAvis
!pip install spacy
!pip install nmslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 41.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.4 MB/s eta 0:00:00
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=876213 sha256=55b61f1e6ea01c07c12091d6905fba27ecb64b12bbbe3e36f5a60a0e8fc0c932
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uni

In [3]:
import nltk
nltk.download("stopwords")
import numpy as np
import json
import glob
import scispacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases, Word2Vec
from gensim.models.phrases import Phraser

#Spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.read_csv('/kaggle/input/ctg-studies/LDA_Topic_raw_ctg-studies.csv')
data_title_sm = data['Study Title']

In [5]:
print(data_title_sm)

0                 Clinical Evaluation of Flortaucipir F 18
1        Enhanced, EHR-facilitated Cancer Symptom Contr...
2        CYT107 After Vaccine Treatment (Provenge®) in ...
3        Pharmacokinetics and Safety Study of PT010 and...
4        Is End Tidal CO2 Level Elevation During Upper ...
                               ...                        
26188             Diabetes Prevention Using SMS Technology
26189    Bosutinib Treatment Extension Study Only For S...
26190    A Single Arm Study Evaluating the Efficacy and...
26191                    Gamma tACS in Alzheimer's Disease
26192    Anemia Studies in Chronic Kidney Disease (CKD)...
Name: Study Title, Length: 26193, dtype: object


In [6]:
#stopwords = stopwords.words("english")
nlp = spacy.load("en_core_sci_sm")
nlp_lrg = spacy.load("en_core_sci_lg")

/opt/conda/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [38]:
print(stopwords)
print(nlp)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# Function to preprocess and lemmatize text
def lemmatize_title_sm(text):
    # Convert the text to a spaCy Doc object
    doc_title_sm = nlp(text)
    
    # Lemmatize the text
    lemmas_title_sm = [token.lemma_ for token in doc_title_sm if not token.is_stop and not token.is_punct]
    
    return ' '.join(lemmas_title_sm)

In [9]:
# Apply the function to the 'study title' column
data_lemma_title_sm = data_title_sm.apply(lemmatize_title_sm)

In [10]:
print(data_lemma_title_sm)

0                    clinical evaluation Flortaucipir f 18
1        enhance ehr-facilitated Cancer Symptom Control...
2        cyt107 Vaccine treatment provenge ® patient Me...
3        Pharmacokinetics Safety study pt010 pt003 heal...
4        end tidal co2 level Elevation Upper Endoscopy ...
                               ...                        
26188                   Diabetes Prevention SMS Technology
26189    Bosutinib Treatment extension Study subject ch...
26190    single arm Study evaluate Efficacy Safety Pral...
26191                         gamma tacs Alzheimer Disease
26192    anemia Studies Chronic Kidney Disease ckd eryt...
Name: Study Title, Length: 26193, dtype: object


In [13]:
# Preprocess the text
data_words_title_sm = data_lemma_title_sm.apply(lambda x: x.split())

# Create a dictionary and corpus
id2word_title_sm = corpora.Dictionary(data_words_title_sm)
corpus_title_sm = [id2word_title_sm.doc2bow(text) for text in data_words_title_sm]

In [14]:
print(data_words_title_sm)

0              [clinical, evaluation, Flortaucipir, f, 18]
1        [enhance, ehr-facilitated, Cancer, Symptom, Co...
2        [cyt107, Vaccine, treatment, provenge, ®, pati...
3        [Pharmacokinetics, Safety, study, pt010, pt003...
4        [end, tidal, co2, level, Elevation, Upper, End...
                               ...                        
26188              [Diabetes, Prevention, SMS, Technology]
26189    [Bosutinib, Treatment, extension, Study, subje...
26190    [single, arm, Study, evaluate, Efficacy, Safet...
26191                    [gamma, tacs, Alzheimer, Disease]
26192    [anemia, Studies, Chronic, Kidney, Disease, ck...
Name: Study Title, Length: 26193, dtype: object


In [15]:
print(id2word_title_sm)

Dictionary<31461 unique tokens: ['18', 'Flortaucipir', 'clinical', 'evaluation', 'f']...>


In [52]:
# Train the LDA model
lda_model_title_sm = gensim.models.LdaModel(corpus=corpus_title_sm,
                                   id2word=id2word_title_sm,
                                   num_topics=20, 
                                   random_state=100,
                                   update_every=1,
                                   chunksize=100,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=True)


In [54]:
# Visualize the results
lda_display_title_sm = gensimvis.prepare(lda_model_title_sm, corpus_title_sm, id2word_title_sm, sort_topics=False)
pyLDAvis.display(lda_display_title_sm)

In [38]:

pyLDAvis.save_html(lda_display_title_sm, 'lda_title_sm.html')

NameError: name 'lda_display_title_sm' is not defined

In [ ]:
# Using briefSummary column

In [17]:
data_summary = data['Brief Summary']

In [16]:
# Function to preprocess and lemmatize text
def lemmatize_summary(text):
    # Convert the text to a spaCy Doc object
    doc_summary = nlp(text)
    
    # Lemmatize the text
    lemmas_summary = [token.lemma_ for token in doc_summary if not token.is_stop and not token.is_punct]
    
    return ' '.join(lemmas_summary)

In [18]:
# Apply the function to the 'study title' column
data_lemma_summary= data_summary.apply(lemmatize_summary)

In [14]:
print(data_lemma_summary)

0        study design expand database flortaucipir f 18...
1        early phase trial investigate enhance electron...
2        randomized phase ii trial study glycosylate re...
3        study assess pharmacokinetic safety dose pt010...
4        course endoscopic procedure air historically i...
                               ...                        
26188    type 2 diabete major healthcare problem develo...
26189    objective study provide long term access bosut...
26190    single arm open-label multi-center study desig...
26191    brain oscillation ubiquitous human brain impli...
26192    phase 3 study hemodialysis-dependent subject a...
Name: Brief Summary, Length: 26193, dtype: object


In [23]:
# Preprocess the text
data_words_summary = data_lemma_summary.apply(lambda x: x.split())

# Create a dictionary and corpus
id2word_summary = corpora.Dictionary(data_words_summary)
corpus_summary = [id2word_summary.doc2bow(text) for text in data_words_summary]

In [46]:
print(data_words_summary)

0        [study, design, expand, database, flortaucipir...
1        [early, phase, trial, investigate, enhance, el...
2        [randomized, phase, ii, trial, study, glycosyl...
3        [study, assess, pharmacokinetic, safety, dose,...
4        [course, endoscopic, procedure, air, historica...
                               ...                        
26188    [type, 2, diabete, major, healthcare, problem,...
26189    [objective, study, provide, long, term, access...
26190    [single, arm, open-label, multi-center, study,...
26191    [brain, oscillation, ubiquitous, human, brain,...
26192    [phase, 3, study, hemodialysis-dependent, subj...
Name: Brief Summary, Length: 26193, dtype: object


In [18]:
print(id2word_summary)

Dictionary<63668 unique tokens: ['18', 'aging', 'analysis', 'binding', 'brain']...>


In [19]:
# Train the LDA model
lda_model_summary = gensim.models.LdaModel(corpus=corpus_summary,
                                   id2word=id2word_summary,
                                   num_topics=10, 
                                   random_state=100,
                                   update_every=1,
                                   chunksize=100,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=True)


In [21]:
# Visualize the results
lda_display_summary = gensimvis.prepare(lda_model_summary, corpus_summary, id2word_summary, sort_topics=False)
pyLDAvis.display(lda_display_summary)

In [ ]:
pyLDAvis.save_html(lda_display_summary, 'lda_summary.html')

In [ ]:
#Large model LDA


In [43]:
# Function to preprocess and lemmatize text
def lemmatize_lrg(text):
    # Convert the text to a spaCy Doc object
    doc = nlp_lrg(text)
    
    # Lemmatize the text
    lemmas_lrg = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return ' '.join(lemmas_lrg)

In [ ]:
# Apply the function to the 'study title' column
data_lemma_lrg = data_title.apply(lemmatize_lrg)

In [46]:
# Preprocess the text
data_words_lrg = data_lemma_lrg.apply(lambda x: x.split())

# Create a dictionary and corpus
id2word_lrg = corpora.Dictionary(data_words_lrg)
corpus_lrg = [id2word_lrg.doc2bow(text) for text in data_words_lrg]

In [48]:
print(data_words_lrg)

0              [clinical, evaluation, flortaucipir, f, 18]
1        [enhance, ehr-facilitated, Cancer, Symptom, co...
2        [cyt107, vaccine, Treatment, Provenge, ®, pati...
3        [pharmacokinetic, safety, study, pt010, pt003,...
4        [end, tidal, co2, level, elevation, Upper, End...
                               ...                        
26188              [Diabetes, Prevention, SMS, Technology]
26189    [bosutinib, treatment, extension, study, subje...
26190    [single, Arm, study, evaluate, Efficacy, safet...
26191                    [gamma, tacs, alzheimer, disease]
26192    [anemia, study, chronic, Kidney, disease, ckd,...
Name: Study Title, Length: 26193, dtype: object


In [55]:
# Train the LDA model
lda_model_lrg = gensim.models.LdaModel(corpus=corpus_lrg,
                                   id2word=id2word_lrg,
                                   num_topics=15, 
                                   random_state=100,
                                   update_every=1,
                                   chunksize=100,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=False)


In [56]:
# Visualize the results
lda_display_lrg = gensimvis.prepare(lda_model_lrg, corpus_lrg, id2word_lrg, sort_topics=False)
pyLDAvis.display(lda_display_lrg)

In [ ]:
pyLDAvis.save_html(lda_display, 'lda_lrg.html')

In [ ]:
#bigram Title

In [26]:
# Create bigrams
bigram = Phrases(data_words_title_sm, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = Phraser(bigram)
data_words_bigrams_title = [bigram_mod[doc] for doc in data_words_title_sm]

In [ ]:
print(data_words_bigrams_title)

In [29]:
# Train Word2Vec model
model = Word2Vec(data_words_bigrams_title, vector_size=100, window=5, min_count=1, workers=4)

# Use the trained Word2Vec model to replace words in data_words_bigrams with their corresponding word vectors
data_vectors_bigram = [[model.wv[word] for word in doc] for doc in data_words_bigrams_title]


In [ ]:
print(data_vectors_bigram)

In [32]:
def clean_text(text):
    return text.encode('utf-8', errors='ignore').decode('utf-8')

# Apply the clean_text function to each document in your data
cleaned_data_bigram_title = [[clean_text(word) for word in doc] for doc in data_words_bigrams_title]

# Create a dictionary and corpus using the cleaned data
id2word_bigram = corpora.Dictionary(cleaned_data_bigram_title)
corpus_bigram = [id2word_bigram.doc2bow(text) for text in cleaned_data_bigram_title]

In [102]:
# Train the LDA model
lda_model_bigram_title = gensim.models.LdaModel(corpus=corpus_bigram,
                                   id2word=id2word_bigram,
                                   num_topics=15, 
                                   random_state=100,
                                   update_every=1,
                                   chunksize=100,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=False)


In [103]:
# Visualize the results
lda_display_bigram_title = gensimvis.prepare(lda_model_bigram_title, corpus_bigram, id2word_bigram, sort_topics=False)
pyLDAvis.display(lda_display_bigram_title)

In [104]:
pyLDAvis.save_html(lda_display_bigram_title, 'lda_bigram_title.html')

In [ ]:
#Bigram Summary#NOT currently working#

In [ ]:
data_summary = data['Brief Summary']
print(data_words_summary)

In [ ]:
# Clean data
cleaned_data_summary = [[re.sub(r'\W|\d', '', word) for word in doc] for doc in data_words_summary]

# Print cleaned data
print(cleaned_data_summary)

In [96]:
def clean_text_bigram_summary(text):
    return text.encode('utf-8', errors='ignore').decode('utf-8')
# Apply the clean_text function to each document in your data
cleaned_data_bigram_summary = [[clean_text(word) for word in doc] for doc in data_words_bigrams_summary_cleaned]
data_vectors_bigram_summary_cleaned = [[word.decode('utf-8', 'ignore') if isinstance(word, bytes) else word for word in doc] for doc in data_vectors_bigram_summary_cleaned]

In [89]:
# Create bigrams
bigram_summary_cleaned = Phrases(cleaned_data_bigram_summary, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod_summary = Phraser(bigram_summary_cleaned)

data_words_bigrams_summary_cleaned = [bigram_mod_summary[doc] for doc in cleaned_data_bigram_summary]
#bigram_summary = gensim.models.Phrases(cleaned_data_summary, min_count=5, threshold=50)
#bigram_mod_summary = gensim.models.phrases.Phraser(bigram_summary_cleaned)
#data_words_bigrams_summary_cleaned = [bigram_mod_summary[doc] for doc in cleaned_data_summary]

In [ ]:
print(data_words_bigrams_summary_cleaned)

In [90]:
# Create Word2Vec model
model_summary = Word2Vec(data_words_bigrams_summary_cleaned, vector_size=100, window=5, min_count=1, workers=4)


In [93]:
# Use the trained Word2Vec model to replace words in data_words_bigrams with their corresponding word vectors
#data_vectors_bigram_summary_cleaned = [[model_summary.wv[word] for word in doc] for doc in cleaned_data_bigram_summary]
data_vectors_bigram_summary_cleaned = [[model_summary.wv[word] for word in doc if word in model_summary.wv.key_to_index] for doc in cleaned_data_bigram_summary]

In [97]:
# Create a dictionary and corpus using the cleaned data
id2word_bigram_summary = corpora.Dictionary(data_vectors_bigram_summary_cleaned)
corpus_bigram_summary = [id2word_bigram_summary.doc2bow(text) for text in data_vectors_bigram_summary_cleaned]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 0: invalid start byte

In [ ]:
# Train the LDA model
lda_model_bigram_title = gensim.models.LdaModel(corpus=corpus_bigram,
                                   id2word=id2word_bigram,
                                   num_topics=15, 
                                   random_state=100,
                                   update_every=1,
                                   chunksize=100,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=False)


In [ ]:
# Visualize the results
lda_display_bigram_title = gensimvis.prepare(lda_model_bigram_title, corpus_bigram, id2word_bigram, sort_topics=False)
pyLDAvis.display(lda_display_bigram_title)